<a href="https://colab.research.google.com/github/DmitriiKnyazkin/Yolo_TEST/blob/master/MP_ver_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task


In [4]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from google.colab.patches import cv2_imshow

In [5]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [11]:
cap = cv2.VideoCapture('/content/video_in_3.mp4')
imageWidth = cap.get(3)# Разрешение входного файла, ширина
imageHeight = cap.get(4)# Разрешение входного файла, высота
fps = cap.get(cv2.CAP_PROP_FPS)  # Частота входного файла, FPS
len_freame = int(cap.get(7)+1) # Длина выходного файла, кадры
output_width = 640 # Разрешение выходного файла, ширина
output_height = 480 # Разрешение выходного файла, высота

writer = cv2.VideoWriter("/content/video_out_3.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (output_width, output_height))

for i in range(len_freame):
  cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
  res, frame = cap.read()
  mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
  detection_result = detector.detect(mp_image)
  detection_result.face_landmarks[0][6].y
  detection_result.face_landmarks[0][6].x
  #Высчитываем координаты кадра относительно ценра глаз (точка 6)
  y_zenter = detection_result.face_landmarks[0][6].y * imageHeight
  x_zenter = detection_result.face_landmarks[0][6].x * imageWidth
  y_0 = int(y_zenter - output_height/2)
  y_1 = y_0 + output_height
  x_0 = int(x_zenter - output_width/2)
  x_1 = x_0 + output_width
  frame_cut_out = frame[y_0:y_1, x_0:x_1]
  writer.write(frame_cut_out)

writer.release()

IndexError: list index out of range

In [12]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 41.8 MB/s eta 0:00:00


In [16]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
# create a DataFrame


Mounted at /content/drive


In [ ]:
print(df)

In [ ]:
df = pd.DataFrame(data)
print(df)
df.to_csv('/content/drive/My Drive/mydata.csv', index=False)

In [18]:
df = pd.DataFrame({'x1', 'x2', 'y1', 'y2'})
print(df)

    0
0  x1
1  y1
2  x2
3  y2


In [14]:
from ultralytics import YOLO


# Загрузка модели YOLOv8
model = YOLO('/content/1280_200.pt') #YOLO('yolov8n.pt')

# Список цветов для различных классов
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]

# Открытие исходного видеофайла
input_video_path = 'video_in_4.mp4'
capture = cv2.VideoCapture(input_video_path)

# Чтение параметров видео
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Настройка выходного файла
output_video_path = 'video_in_detect_4.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while True:
    # Захват кадра
    ret, frame = capture.read()
    if not ret:
        break

    # Обработка кадра с помощью модели YOLO
    results = model(frame)[0]

    # Получение данных об объектах
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)
    # Рисование рамок и подписей на кадре
    for class_id, box, conf in zip(classes, boxes, results.boxes.conf):
        if conf>0.5:
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]
            x1, y1, x2, y2 = box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Запись обработанного кадра в выходной файл
    writer.write(frame)

# Освобождение ресурсов и закрытие окон
capture.release()
writer.release()


0: 736x1280 2 iriss, 2 eyes, 726.4ms
Speed: 14.3ms preprocess, 726.4ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 784.2ms
Speed: 15.4ms preprocess, 784.2ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 794.8ms
Speed: 15.3ms preprocess, 794.8ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 838.6ms
Speed: 15.4ms preprocess, 838.6ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 688.4ms
Speed: 15.6ms preprocess, 688.4ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 516.5ms
Speed: 11.3ms preprocess, 516.5ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 524.3ms
Speed: 12.3ms preprocess, 524.3ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 2 iriss, 2 eyes, 519.